In [1]:
from curses import init_color
import os
import os.path as osp
import copy
import sys
import time 
from datetime import timedelta
from datetime import datetime
import random
import time
from tqdm import tqdm
from collections import Counter

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import argparse
import pickle
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader

from models.AutoEncoder import AutoEncoder_RGB, AutoEncoder_Depth
from models.AutoEncoder import AutoEncoder_Intergrated_Basic, AutoEncoder_Intergrated_Proposed
from models.Network import Face_Detection_Model
from dataloader.dataloader import load_dataset, load_test_dataset
from utility import draw_train_and_test_loss, draw_train_and_test_loss1, draw_accuracy_and_f1_during_training, draw_accuracy_and_f1_during_training1

print(torch.__version__,torch.cuda.is_available(),torch.cuda.device_count())
copy_before_model = True
server = 'server1'
print(f'server : {server} , copy_before_model : {copy_before_model}')

1.10.1+cu102 True 2
server : server1 , copy_before_model : True


In [2]:
# def seed_worker(worker_id):
#     worker_seed = torch.initial_seed() % 2**32
#     np.random.seed(worker_seed)
#     random.seed(worker_seed)

def booltype(str):
    if isinstance(str, bool):
        return str
    if str.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif str.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentError("Boolean value expected")

# def model_save(model, epoch, optimizer, train_loss, val_loss, train_f1, valid_f1, path) :
#     torch.save({
#                 'model_state_dict': model.state_dict(),
#                 'epoch': epoch,
#                 'optimizer_state_dict': optimizer.state_dict(),
#                 'train_loss': train_loss,
#                 'val_loss' : val_loss,
#                 'train_f1' : train_f1,
#                 'val_f1' : valid_f1
#                 }, path)
#     print('Model Save ! > ', path)

In [3]:
# args option
parser = argparse.ArgumentParser(description='face anto-spoofing')
parser.add_argument('--batchsize', default=4, type=int, help='batch size')
parser.add_argument('--workers', default=4, type=int, help='number of workers, 4')
parser.add_argument('--epochs', default=100, type=int, help='train epochs')        
parser.add_argument('--trainratio', default=1.0, type=float, help='ratio to divide train dataset')                               
parser.add_argument('--lr', default=1e-3, type=float, help='learning rate(default: 0.001)')
parser.add_argument('--skf', default=0, type=int, help='stratified k-fold')
parser.add_argument('--attacktype', default='prm', type=str, help='Kinds of Presentation Attacks: r, p, m, prm')
parser.add_argument('--dataset', default=12, type=int, help='dataset type: 12 or 15')
parser.add_argument('--model', default='', type=str, help='rgb, rgbd, rgbp, rgbdp')   
parser.add_argument('--inputchannel', default=3, type=int, help='inputchannel')
parser.add_argument('--crop', default=False, type=booltype, help='use crop (default: False)')
parser.add_argument('--ae-path', default='', type=str, help='Pretrained AutoEncoder path')
parser.add_argument('--save-path', default='./hm_save_file', type=str, help='train logs path')
parser.add_argument('--model-path', default='', type=str, help='model parameter path')
# parser.add_argument('--init-path', default='', type=str, help='model init parameter path')
# parser.add_argument('--message', default='', type=str, help='parameter file name')                     
parser.add_argument('--seed', default=1, type=int, help='Seed for random number generator')
parser.add_argument('--cuda', default=0, type=int, help='gpu number')                                         
parser.add_argument('--device', default='', type=str, help='device when cuda is available')                       
args = parser.parse_known_args()[0]

In [4]:
def fix_seed(random_seed):
    """
    fix seed to control any randomness from a code 
    (enable stability of the experiments' results.)
    """
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
#     torch.use_deterministic_algorithms(True)
    np.random.seed(random_seed)
    random.seed(random_seed)
    os.environ['PYTHONHASHSEED'] = str(random_seed)
#     os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    
fix_seed(args.seed)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

In [5]:
args.inputchannel = 6
args.model = 'rgbp'
args.epochs = 10
args.device = 0

In [6]:
if args.model == 'rgb':
    args.inputchannel = 3
elif args.model == 'rgbd':
    args.inputchannel = 4
elif args.model == 'rgbp':
    args.inputchannel = 6
elif args.model == 'rgbdp':
    args.inputchannel = 7
else:
    print("You need to checkout option 'model' [rgb, rgbd, rgbp, rgbpc]")
    sys.exit(0)

In [7]:
# cuda 관련 코드
use_cuda = True if torch.cuda.is_available() else False
if use_cuda : 
    if args.cuda == 0:
        args.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    elif args.cuda == 1:
        args.device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
    print('device :', args.device)

device : cuda:0


In [8]:
train_dataset, test_dataset = load_dataset(args)
outdoor_testset = load_test_dataset(args, "2. Outdoor")
indoor_dark_testset = load_test_dataset(args, "3. Indoor_dark")

train_loader = DataLoader(train_dataset, batch_size=args.batchsize, shuffle=False, 
                          num_workers=args.workers, pin_memory=True,
                          worker_init_fn=seed_worker,generator=g)
test_loader = DataLoader(test_dataset, batch_size=args.batchsize, shuffle=False, 
                         num_workers=args.workers, pin_memory=True,
                         worker_init_fn=seed_worker,generator=g)
outdoor_loader = DataLoader(outdoor_testset, batch_size=args.batchsize, shuffle=False, 
                            num_workers=args.workers, pin_memory=True,
                            worker_init_fn=seed_worker,generator=g)
dark_loader = DataLoader(indoor_dark_testset, batch_size=args.batchsize, shuffle=False,
                         num_workers=args.workers, pin_memory=True,
                         worker_init_fn=seed_worker,generator=g)

720
240
960
960


In [9]:
for i in train_dataset :
    break

In [10]:
i[0].shape
i[1].shape

(3, 180, 180)

In [79]:
class twostep_model(nn.Module):
    def __init__(self,device):
        super().__init__()
        # Model 생성 및 아키텍쳐 출력   
        self.rgb_model = Face_Detection_Model(3,get_features=True).to(device)    
        self.pt_model = Face_Detection_Model(3,get_features=True).to(device)    
        self.fc = nn.Linear(1024,1).to(device)
        
    def forward(self,rgb,cloud):
        rgb_feature = self.rgb_model(rgb)
        cloud_feature = self.pt_model(cloud)
        features = torch.concat([rgb_feature,cloud_feature],axis=1)
        logit = self.fc(features)
        return logit
        
        


In [80]:
3/0

ZeroDivisionError: division by zero

In [81]:
model = twostep_model(device=args.device)

In [82]:
model = twostep_model(device=args.device)

# Loss, 옵티마이저, 스케줄러 생성 
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

# scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0)

# Train 
train_performs = {'ACC':[],'F1':[]}
test_performs = {'ACC':[],'F1':[], 'Info':[]}
outdoor_performs = {'ACC':[],'F1':[], 'Info':[]}
dark_performs = {'ACC':[],'F1':[], 'Info':[]}

# Variables 
total_train_loss = []
total_test_loss = []
total_outdoor_loss = []
total_dark_loss = []

best_test_f1 = 0
best_test_epoch = 0   
best_test_accu = 0 

epochs = args.epochs + 1
sigmoid = nn.Sigmoid()
loss_fn = nn.BCEWithLogitsLoss()





indoor_check = {'train_loss':[],'test_loss':[],'train_f1':[],'test_f1':[],'train_acc':[],'test_acc':[]}
outdoor_check  = {'test_loss':[],'test_f1':[],'test_acc':[]}
dark_check = {'test_loss':[],'test_f1':[],'test_acc':[]}

for epoch in range(1, epochs) :
    model.train()
    train_loss = []
    train_probs, train_labels = [],[]
    train_bar = tqdm(enumerate(train_loader))
    for step, data in train_bar :  
        rgb, cloud, depth, label = data
        rgb = rgb.float().to(args.device)
        cloud = cloud.float().to(args.device)
        depth = depth.float().to(args.device)
        label = label.float().to(args.device)

        features = rgb
        if args.model == "rgbd":
            features = torch.cat([rgb, depth], dim=1)
        elif args.model == "rgbp":
            features = torch.cat([rgb, cloud], dim=1)
        elif args.model == "rgbdp":
            features = torch.cat([rgb, depth, cloud], dim=1)  

        optimizer.zero_grad()

#         features = torch.cat([rgb, cloud], dim=1)
        logits = model(rgb,cloud)

        logits = logits[:,0]  # logtit (4.2) 텐서이던데 첫째는 batch 이겠고. 근데 왜 0번째 값만 쓸까?
        loss = loss_fn(logits, label.float())
        loss.backward()
        optimizer.step()

        probs = sigmoid(logits)
        train_loss.append(loss.item())
        train_probs += probs.cpu().detach().tolist()
        train_labels += label.cpu().detach().tolist()

        train_bar.set_description("[Train] Epoch[{}/{}][{}/{}] Loss:{} Loss(mean):{}".format(epoch,epochs-1,step+1,len(train_loader),
                                                    round(loss.item(),5),round(np.array(train_loss).mean(),10)
                                                            ))

    train_loss_mean = round(np.array(train_loss).mean(), 10)        

    # scheduler.step()
    print("[Train] Epoch[{}/{}][{}/{}] Loss:{} Loss(mean):{}".format(epoch,epochs-1,step+1,len(train_loader),
                                                    round(loss.item(),5),train_loss_mean))    
    total_train_loss.append(train_loss_mean)
    train_acc = accuracy_score(np.array(train_labels), np.round(train_probs))
    train_f1 = f1_score(np.array(train_labels), np.round(train_probs), average='macro')
    print(f'Train Accuracy : {train_acc:.4f}')
    print(f'Train F1-score : {train_f1:.4f}')
    train_performs['ACC'].append(train_acc)
    train_performs['F1'].append(train_f1)
    print(f'  > Counter(train_labels) : {Counter(train_labels)}')
    print(f'  > Counter(train_preds) : {Counter(np.round(train_probs))}')
    cf = confusion_matrix(np.array(train_labels), np.round(train_probs))
    cf = pd.DataFrame(cf)
    cf.columns = ['Predicted:0','Predicted:1']
    cf.index = ['Label:0','Label:1']    
    print(' --- [Train] Confustion_Matrix & Classification_Report --- ')

    # display(cf)
    print(cf.to_string())
    report = classification_report(np.array(train_labels), np.round(train_probs))
    print(report)

    indoor_check['train_loss'].append(np.array(train_loss).mean())
    indoor_check['train_f1'].append(train_f1)
    indoor_check['train_acc'].append(train_acc)

    model.eval()
    test_loss = []
    test_probs, test_labels = [],[]

    # Indoor 
    test_bar = tqdm(enumerate(test_loader))
    for step, data in test_bar :  
        rgb, cloud, depth, label = data
        rgb = rgb.float().to(args.device)
        cloud = cloud.float().to(args.device)
        depth = depth.float().to(args.device)
        label = label.float().to(args.device)

        features = rgb
        if args.model == "rgbd":
            features = torch.cat([rgb, depth], dim=1)
        elif args.model == "rgbp":
            features = torch.cat([rgb, cloud], dim=1)
        elif args.model == "rgbdp":
            features = torch.cat([rgb, depth, cloud], dim=1)  

        logits,_ = model(features)
        logits = logits[:,0]
        loss = loss_fn(logits, label.float())

        probs = sigmoid(logits)
        test_loss.append(loss.item())
        test_probs += probs.cpu().detach().tolist()
        test_labels += label.cpu().detach().tolist()        
        test_bar.set_description("[Test] Epoch[{}/{}][{}/{}] Loss:{} Loss(mean):{}".format(epoch,epochs-1,step+1,len(test_loader),
                                                    round(loss.item(),5),round(np.array(test_loss).mean(),5)
                                                            ))

    indoor_check['test_loss'].append(np.array(test_loss).mean())
    indoor_check['test_f1'].append(f1_score(np.array(test_labels), np.round(test_probs), average='macro'))
    indoor_check['test_acc'].append(accuracy_score(np.array(test_labels), np.round(test_probs)))

    outdoor_loss = []
    outdoor_probs, outdoor_labels = [], []
    
    break

[Train] Epoch[1/10][125/180] Loss:1.28773 Loss(mean):0.5644163179: : 125it [00:07, 17.82it/s]


KeyboardInterrupt: 

In [72]:
model

twostep_model(
  (rgb_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

In [73]:
args.device

device(type='cuda', index=0)